# Writing data to and reading data from a database using Python

## Libraries and settings

In [ ]:
# Libraries
import os
import sqlite3
import fnmatch
import pandas as pd
import matplotlib.pyplot as plt

# Define settings for graphics
# plt.style.use('dark_background')

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

## Create sqlite data base

In [ ]:
# Create data base
conn = sqlite3.connect('apartment_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

## Create empty SQL-table in the database

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS apartments_table (OrderId VARCHAR(50),
                                                                Rooms DECIMAL(8,2),
                                                                Area INT(8),
                                                                Price DECIMAL(8,2))''')
# Confirm changes to the table
conn.commit()

## Read data from file to data frame

In [ ]:
# Read data
df = pd.read_csv('apartments_data_prepared.csv', 
                  sep=',', 
                  encoding='utf-8')[['web-scraper-order', 'rooms', 'area', 'price']]

# Calculate new variable
df['price_per_m2'] = df['price'] / df['area']

print(df.shape)
df.head(5)

## Write data to the SQL-table in data base

In [ ]:
df.to_sql(name = 'apartments_table',
          con = conn,
          index = False,
          if_exists = 'replace')

## Query the SQL-table

In [ ]:
# Query the SQL-table
cursor.execute('''SELECT * FROM apartments_table 
                  WHERE Rooms >= 2.0 
                  AND Price > 1000''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['OrderId','Rooms','Area','Price', 'Price_per_m2'])    
print(df)

## Plot histogram of rental prices

In [ ]:
plt.figure(figsize=(6,4))
df['Price_per_m2'].plot.hist(grid=True, 
                   bins=20, 
                   rwidth=0.9,
                   color='greenyellow')
plt.title('Appartment price per m2 (CHF)')
plt.xlabel('Price per m2')
plt.ylabel('Frequency')
plt.grid(linestyle='-', linewidth=0.1)

conn.close()

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')